In [0]:
#get the file name from the ADF
fileName = dbutils.widgets.get('fileName')
#fileName = 'Product.csv'
#splitting the filename with '.' and getting the first word of the file
fileNameWithoutExt = fileName.split('.')[0]
print(fileNameWithoutExt)

Product


In [0]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from datetime import datetime as dt

#change all values based on the azure credentials
sqlDbName = 'sql_sample'
dbUserName = 'vignan'
passwordKey = 'dbServerPassword'
stgAccountSAStokenKey = 'project2SAStoken'
landingFileName = fileName
databricksScopeName = 'scope_secondary'
dbServer = 'key'
dbServerPortNumber = '1433'
storageContainer = 'project2landing'
storageAccount = 'practiceprojects'
landingMountPoint = '/mnt'

In [0]:
#mounting
if not any(mount.mountPoint == landingMountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(source='wasbs://{}@{}.blob.core.windows.net'.format(storageContainer, storageAccount), mount_point=landingMountPoint, extra_configs={'fs.azure.sas.{}.{}.blob.core.windows.net'.format(storageContainer,storageAccount):dbutils.secrets.get(scope=databricksScopeName, key=stgAccountSAStokenKey)})
    print('Mounted Successfully')
else:
    print('Already mounted')

Already mounted


In [0]:
#connecting to azure sql db
dbPassword = dbutils.secrets.get(scope= databricksScopeName, key = passwordKey)
serverurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbServerPortNumber, sqlDbName, dbUserName)
connectionProperties = {'password': dbPassword, 'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver'}
df = spark.read.jdbc(url=serverurl, table = 'dbo.FileDetailsFormat', properties = connectionProperties)
display(df)


FileNo,FileName,ColumnName,ColumnDateFormat,ColumnIsNull,ModifiedDate
1,Product,StartDate,MM-dd-yyyy,true,2012-06-18T22:34:09Z
1,Product,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,StartDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,ActiveDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z


In [0]:
df1 = spark.read.csv('/mnt/landing/'+fileName, header=True, inferSchema=True)
display(df1)

#rule
errorFlag = False
errorMessage = ''
totalCount = df1.count()
print(totalCount)
distinctCount = df1.distinct().count()
print(distinctCount)
if distinctCount != totalCount:
    errorFlag = True
    errorMessage = 'Duplications found in the file. Rule 1 Failed'
print(errorMessage)

#Rule2
df2 = df.filter(df.FileName == fileNameWithoutExt).select('ColumnName','ColumnDateFormat')
rows = df2.collect()
for r in rows:
    colName = r[0]
    colFormat = r[1]
    #display(df1.filter(F.to_date(colName, colFormat).isNull()==True))
    formatCount = df1.filter(F.to_date(colName, colFormat).isNotNull()==True).count()
    print(formatCount)
    if formatCount == totalCount:
        errorFlag = True
        errorMessage = errorMessage + 'DateFormat is incorrect for {} '.format(colName)
    else:
        print('All rows are good for column '+ str(colName))
print(totalCount,formatCount)
print(errorMessage)


if errorFlag:
    dbutils.fs.mv('/mnt/landing/'+fileName, '/mnt/rejected/'+fileName)
    dbutils.notebook.exit('{"errorFlag":"true", "errorMessage":"'+errorMessage+'","totalCount":"'+str(totalCount)+'","formatCount":"'+str(formatCount)+'"}')
else:
    dbutils.fs.mv('/mnt/landing/'+fileName, '/mnt/staging/'+fileName)
    dbutils.notebook.exit('{"errorFlag":"false", "errorMessage":"No errors found"}')
